In [1]:
!pip install konlpy

     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     --------------------------------------- 0.0/19.4 MB 435.7 kB/s eta 0:00:45
     --------------------------------------- 0.1/19.4 MB 751.6 kB/s eta 0:00:26
     ----- ---------------------------------- 2.5/19.4 MB 15.8 MB/s eta 0:00:02
     ------------- -------------------------- 6.7/19.4 MB 30.8 MB/s eta 0:00:01
     -------------------- ------------------ 10.1/19.4 MB 38.0 MB/s eta 0:00:01
     --------------------------- ----------- 13.6/19.4 MB 73.1 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 M

In [2]:
!pip install Twitter

  Using cached twitter-1.19.6-py2.py3-none-any.whl (50 kB)


In [3]:
!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [4]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ab/b0/d58dc405fd60ab546ca714321235dc2d455b2dc06bfb4fc1092940c749fc/gensim-4.3.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.7.0 from https://files.pythonhosted.org/packages/70/03/485f73046134400ea25d3cb178c5e6728f9b165f79d09638ecb44ee0e9b1/scipy-1.11.2-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.1 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 775.6 kB/s eta 0:00:00
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.0 MB ? eta -:--

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
theano-pymc 1.1.2 requires filelock, which is not installed.
arviz 0.12.1 requires matplotlib>=3.0, which is not installed.
arviz 0.12.1 requires xarray>=0.16.1, which is not installed.
xarray-einstats 0.5.1 requires xarray>=2022.09.0, which is not installed.


In [5]:
import os
import pymysql
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec

In [6]:
# 본인 컴퓨터의 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [7]:
os.environ["JAVA_HOME"]

'C:\\Program Files\\Java\\jdk-11'

In [8]:
from konlpy.tag import Twitter

In [9]:
# 한국어 문자열을 단어로 나누고 품사를 리턴하는 Twitter() 객체 생성
twitter = Twitter()

c:\Users\tjk\AppData\Local\anaconda3\envs\movie_env01\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [10]:
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워")

[('안녕', 'Noun'),
 ('하십니까', 'Verb'),
 ('좋은', 'Adjective'),
 ('아침', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반가워', 'Adjective')]

In [11]:
# stem=True: 동사인 하십니까 반가워는 기본 동사형 '안녕하다'와 '반갑다'로 변환
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워 집으로 뛰어 갔다 정말 참으로 좋다", stem=True)

[('안녕', 'Noun'),
 ('하다', 'Verb'),
 ('좋다', 'Adjective'),
 ('아침', 'Noun'),
 ('이다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반갑다', 'Adjective'),
 ('집', 'Noun'),
 ('으로', 'Josa'),
 ('뛰다', 'Verb'),
 ('가다', 'Verb'),
 ('정말', 'Noun'),
 ('참으로', 'Noun'),
 ('좋다', 'Adjective')]

In [13]:
# 텍스트 전처리 함수
# text는 전처리할 영화 줄거리(synopsys)입니다.
def preprocessingText(text):
    # 전처리 결과를 저장할 리스트
    stems = []
    
    # text를 단어로 나누고 품사를 붙여서 리턴
    tagged_review = twitter.pos(text, stem=True)
    
    # tagged_review에서 하나를 꺼내서 word, pos에 대입
    # 단어는 word에, 품사는 pos에 대입
    for word, pos in tagged_review:
        if pos == "Noun" or pos == "Adjective":
            stems.append(word)
            
    return " ".join(stems)

In [14]:
# 함수 테스트
preprocessingText("아버지가 방에 들어가신다 그래서 슬펐다 배가 아파요")

'아버지 방 슬프다 배 아프다'

In [15]:
# DB 연결
db = pymysql.connect(
    host="database-1.crcwt5epkeh5.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    user='admin',
    passwd='password',
    db='movie_db',
    charset='utf8'
)

In [16]:
# DB 연결 테스트
sql = "SELECT * FROM movie_tbl;"
movie_df = pd.read_sql(sql, db)

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\1045968588.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(sql, db)


In [17]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None
...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None


In [18]:
movie_df['synopsys'][0]

"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 뒤, ‘마석도’(마동석)는 새로운 팀원들과 함께 살인사건을 조사한다. 사건 조사 중, ‘마석도’는 신종 마약 사건이 연루되었음을 알게 되고 수사를 확대한다. 한편, 마약 사건의 배후인 '주성철'(이준혁)은 계속해서 판을 키워가고 약을 유통하던 일본 조직과 '리키'(아오키 무네타카)까지 한국에 들어오며 사건의 규모는 점점 더 커져가는데... 나쁜 놈들 잡는 데 이유 없고 제한 없다. 커진 판도 시원하게 싹 쓸어버린다!"

In [19]:
preprocessingText(movie_df['synopsys'][0])

'대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마 석도 마동석 새롭다 팀원 살인 사건 조사 사건 조사 중 마 석도 신종 마약 사건 연루 알 수사 확대 한편 마약 사건 배후 주성철 이준혁 은 계속 판 약 유통 일본 조직 리키 아오키 무 타카 한국 사건 규모 점점 더 나쁘다 놈 데 이유 없다 제한 없다 판도 시원하다 싹'

In [20]:
# "synopsys_clear" 컬럼을 추가하고 초기값으로 np.NaN(결측치) 대입
movie_df["synopsys_clear"] = np.NaN

In [21]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector,synopsys_clear
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None,NaN
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None,NaN
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None,NaN
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None,NaN
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None,NaN
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None,NaN
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None,NaN
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None,NaN


In [22]:
len(movie_df)

627

In [23]:
row_num = len(movie_df)

In [24]:
for index in tqdm(range(row_num)):
    try:
        # df_movie.loc[index, "synopsys"]: synopsys 컬럼의 index번째 줄 데이터를 synopsys에 대입
        synopsys = movie_df.loc[index, "synopsys"]
        
        # df.loc[index, "synopsys_clear"]: Synopsys_clear 컬럼의 index번째 줄에 대입
        movie_df.loc[index, "synopsys_clear"] = preprocessingText(synopsys)
    except Exception as e:
        # 에러 발생 시 결측치 대입
        movie_df.loc[index, "synopsys_clear"] = np.NaN

  0%|          | 0/627 [00:00<?, ?it/s]

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\2672635914.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마 석도 마동석 새롭다 팀원 살인 사건 조사 사건 조사 중 마 석도 신종 마약 사건 연루 알 수사 확대 한편 마약 사건 배후 주성철 이준혁 은 계속 판 약 유통 일본 조직 리키 아오키 무 타카 한국 사건 규모 점점 더 나쁘다 놈 데 이유 없다 제한 없다 판도 시원하다 싹' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  movie_df.loc[index, "synopsys_clear"] = preprocessingText(synopsys)


In [25]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector,synopsys_clear
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None,대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마...
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None,이 근처 폐허 없다 문 있다 규슈 한적하다 마을 살 있다 소녀 스즈메 문 여행 중인...
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None,불 물 공기 흙 개 원소 살 있다 엘리 멘트 시티 재치 있다 불 열정 넘치다 앰버 ...
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None,전국 제패 북산 농구부 인방 꿈 열정 도전 그린 영화
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None,가모라 를 슬픔 있다 피터 퀼 이 위기 처 은하계 동료 위해 다시 한번 가디언즈 팀...
...,...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None,의문 존재 피해 외딴 숲 집 자신 규칙 아내 아들 남자 날 가족 그 집 은신 정체 ...
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None,프랑스 파리 퍼스 널 쇼퍼 일 미국 여자 린 크리스틴 스튜어트 영혼 대화 수 있다 ...
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None,자유롭다 삶 여행가 에바 아들 케빈 그녀 도 일과 양육 동시 에바 케빈 이유 반항 ...
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None,익명 고객 의뢰 일 완수 특별사면 보장 무법 변호사 제안 알제리 감옥 가석방 빅 산...
